# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-8b86b1b429-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Authors comparison explorative search") 

Consider the following exploratory scenario:


>  Investigate the the production of Paul Auster and Ian McEwan, check how many books they have written for each litarature genre, gather information about their production and about their works which are not books



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q47461344`| writtenwork   | node |
| `wd:Q214642`  | Paul Auster   | node |
| `wd:Q190379`  | Ian McEwan    | node |




Also consider

```
wd:Q214642 ?rel ?obj  . 
```
is the BGP to retrieve all **properties of Paul Auster**


The workload should


1. Identify the BGP for obtaining the books (with publishing date and genre) published by the two authors

2. Did the authors published a book in the same year? What is the longest period without publishing a book for the two authors? 

3. Did the authors produced, acted or directed a film? If so, did they write the screenplay?

4. How many films were derived from the books of these two authors? 

5. Which author won more literature-related awards? Have they ever being nominated for a Nobel award? 

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT( ?obj)
WHERE { 
wd:Q214642 ?rel ?obj  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '191')]


1

# Task 1

*Identify the BGP for obtaining the books (with publishing date and genre) published by the two authors*

In [15]:
# Check author properties
queryString = """
SELECT distinct ?rel ?name
where{
    wd:Q214642 ?rel ?obj  .
    ?rel <http://schema.org/name> ?name .
    filter(!regex(?name, "ID"))
}
"""

print("Results")
run_query(queryString)

Results
[('rel', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('rel', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('rel', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('rel', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('rel', 'http://www.wikidata.org/prop/direct/P1317'), ('name', 'floruit')]
[('rel', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('rel', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('rel', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('rel', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('rel', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('rel', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('rel', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birt

35

In [11]:
# Check the notable works 
queryString = """
SELECT distinct ?x ?name
WHERE{
    wd:Q214642 wdt:P800 ?obj .
    ?obj <http://schema.org/name> ?name .
}
"""

print("Results")
run_query(queryString)

Results
[('name', 'The Book of Illusions')]
[('name', 'Man in the Dark')]
[('name', 'The New York Trilogy')]
[('name', 'Leviathan')]


4

In [12]:
# Probably he didn't write only notable works
queryString = """
SELECT distinct ?obj ?name2 ?name ?type
WHERE{
    ?obj wdt:P50 wd:Q214642 .
    ?obj wdt:P31 ?type .
    ?type <http://schema.org/name> ?name .
    ?obj <http://schema.org/name> ?name2 .
}
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1951470'), ('name2', 'Mr. Vertigo'), ('name', 'written work'), ('type', 'http://www.wikidata.org/entity/Q47461344')]
[('obj', 'http://www.wikidata.org/entity/Q492991'), ('name2', 'Timbuktu'), ('name', 'written work'), ('type', 'http://www.wikidata.org/entity/Q47461344')]
[('obj', 'http://www.wikidata.org/entity/Q581485'), ('name2', 'The Book of Illusions'), ('name', 'written work'), ('type', 'http://www.wikidata.org/entity/Q47461344')]
[('obj', 'http://www.wikidata.org/entity/Q581485'), ('name2', 'The Book of Illusions'), ('name', 'book'), ('type', 'http://www.wikidata.org/entity/Q571')]
[('obj', 'http://www.wikidata.org/entity/Q526792'), ('name2', 'Travels in the Scriptorium'), ('name', 'written work'), ('type', 'http://www.wikidata.org/entity/Q47461344')]
[('obj', 'http://www.wikidata.org/entity/Q1817920'), ('name2', 'Man in the Dark'), ('name', 'written work'), ('type', 'http://www.wikidata.org/entity/Q47461344')]
[('obj', 'http://ww

35

So there are not only "written work" but also other types of work that can be considered similar to books. 

In [13]:
# Probably he didn't write only notable works
queryString = """
SELECT distinct ?name ?type
WHERE{
    ?obj wdt:P50 wd:Q214642 .
    ?obj wdt:P31 ?type .
    ?type <http://schema.org/name> ?name .
}
"""

print("Results")
run_query(queryString)

Results
[('name', 'book series'), ('type', 'http://www.wikidata.org/entity/Q277759')]
[('name', 'written work'), ('type', 'http://www.wikidata.org/entity/Q47461344')]
[('name', 'literary work'), ('type', 'http://www.wikidata.org/entity/Q7725634')]
[('name', 'novel'), ('type', 'http://www.wikidata.org/entity/Q8261')]
[('name', 'version, edition, or translation'), ('type', 'http://www.wikidata.org/entity/Q3331189')]
[('name', 'book'), ('type', 'http://www.wikidata.org/entity/Q571')]
[('name', 'short story collection'), ('type', 'http://www.wikidata.org/entity/Q1279564')]
[('name', 'graphic novel'), ('type', 'http://www.wikidata.org/entity/Q725377')]


8

In [23]:
# Book properties (I use book and written work)
queryString = """
SELECT distinct ?name ?ret
where{
    wd:Q214642 wdt:P800 ?obj .
    ?obj ?x ?book .
    ?obj ?ret ?z . 
    ?ret <http://schema.org/name> ?name .
    filter(!regex(?name, "ID"))
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
}
"""

print("Results")
run_query(queryString)

Results
[('name', 'publisher'), ('ret', 'http://www.wikidata.org/prop/direct/P123')]
[('name', 'genre'), ('ret', 'http://www.wikidata.org/prop/direct/P136')]
[('name', 'title'), ('ret', 'http://www.wikidata.org/prop/direct/P1476')]
[('name', 'follows'), ('ret', 'http://www.wikidata.org/prop/direct/P155')]
[('name', 'followed by'), ('ret', 'http://www.wikidata.org/prop/direct/P156')]
[('name', 'OCLC control number'), ('ret', 'http://www.wikidata.org/prop/direct/P243')]
[('name', 'instance of'), ('ret', 'http://www.wikidata.org/prop/direct/P31')]
[('name', 'language of work or name'), ('ret', 'http://www.wikidata.org/prop/direct/P407')]
[('name', 'country of origin'), ('ret', 'http://www.wikidata.org/prop/direct/P495')]
[('name', 'author'), ('ret', 'http://www.wikidata.org/prop/direct/P50')]
[('name', 'publication date'), ('ret', 'http://www.wikidata.org/prop/direct/P577')]
[('name', 'narrative location'), ('ret', 'http://www.wikidata.org/prop/direct/P840')]
[('name', 'ISBN-13'), ('ret',

20

In [24]:
# Paul Auster's books
queryString = """
SELECT * where{
{SELECT distinct ?name ?pubdate (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q214642 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    ?genre1 <http://schema.org/name> ?genre .
} group by ?name ?pubdate}
UNION{
SELECT distinct ?name ?pubdate (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q214642 .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    ?genre1 <http://schema.org/name> ?genre .

}group by ?name ?pubdate}

}
"""

print("Results")
run_query(queryString)

Results
[('name', 'The New York Trilogy'), ('pubdate', '1987-01-01T00:00:00Z'), ('genres', 'novel, mystery fiction')]
[('name', 'Leviathan'), ('pubdate', '1992-01-01T00:00:00Z'), ('genres', 'crime novel, detective fiction, hardboiled')]
[('name', 'The Book of Illusions'), ('pubdate', '2002-01-01T00:00:00Z'), ('genres', 'novel, novel')]
[('name', 'Mr. Vertigo'), ('pubdate', '1994-04-05T00:00:00Z'), ('genres', 'novel')]
[('name', 'The Brooklyn Follies'), ('pubdate', '2005-01-01T00:00:00Z'), ('genres', 'novel')]
[('name', 'The New York Trilogy'), ('pubdate', '1987-01-01T00:00:00Z'), ('genres', 'novel, mystery fiction')]
[('name', 'Leviathan'), ('pubdate', '1992-01-01T00:00:00Z'), ('genres', 'crime novel, detective fiction, hardboiled')]
[('name', 'The Music of Chance'), ('pubdate', '1990-01-01T00:00:00Z'), ('genres', 'novel, hardboiled')]
[('name', 'Invisible'), ('pubdate', '2009-01-01T00:00:00Z'), ('genres', 'novel')]
[('name', 'The Book of Illusions'), ('pubdate', '2002-01-01T00:00:00Z'

15

In [25]:
# ANSWER to 1
queryString = """
SELECT distinct ?name ?pubdate ?genres where{
{SELECT distinct ?name ?pubdate (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q190379 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
} group by ?name ?pubdate}
UNION{
SELECT distinct ?name ?pubdate (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q190379  .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
}group by ?name ?pubdate}
UNION
{SELECT distinct ?name ?pubdate (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q214642 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
} group by ?name ?pubdate}
UNION{
SELECT distinct ?name ?pubdate (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q214642 .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)

}group by ?name ?pubdate}

}order by ?pubdate



"""

print("Results")
run_query(queryString)

Results
[('name', 'The Cement Garden'), ('pubdate', '1978-01-01T00:00:00Z'), ('genres', 'novel')]
[('name', 'The Comfort of Strangers'), ('pubdate', '1981-01-01T00:00:00Z'), ('genres', 'novel')]
[('name', 'The Child in Time'), ('pubdate', '1987-01-01T00:00:00Z'), ('genres', 'novel')]
[('name', 'The New York Trilogy'), ('pubdate', '1987-01-01T00:00:00Z'), ('genres', 'novel, mystery fiction')]
[('name', 'In the Country of Last Things'), ('pubdate', '1987-01-01T00:00:00Z'), ('genres', 'epistolary novel, science fiction novel, dystopian novel')]
[('name', 'The Music of Chance'), ('pubdate', '1990-01-01T00:00:00Z'), ('genres', 'novel, hardboiled')]
[('name', 'Leviathan'), ('pubdate', '1992-01-01T00:00:00Z'), ('genres', 'crime novel, detective fiction, hardboiled')]
[('name', 'Black Dogs'), ('pubdate', '1992-07-01T00:00:00Z'), ('genres', 'novel')]
[('name', 'Mr. Vertigo'), ('pubdate', '1994-04-05T00:00:00Z'), ('genres', 'novel')]
[('name', 'Enduring Love'), ('pubdate', '1997-01-01T00:00:00Z'

23

***

# Task 2

*Did the authors published a book in the same year? What is the longest period without publishing a book for the two authors?*


In [26]:
# Doesn't work with year(date)
queryString = """
ASK WHERE{

SELECT distinct count(?name) ?sss where{
{SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q190379 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
} group by ?name ?sss}
UNION{
SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q190379  .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
}group by ?name ?sss}
UNION
{SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q214642 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
} group by ?name ?sss}
UNION{
SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q214642 .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
}group by ?name ?sss}

} group by ?sss
having(count(?name)>1)

}
"""
#filter(substr(str(?pubdate),1,4)=substr(str(?pubdate1),1,4))

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

In [27]:
# Same a s before but without the ask query
queryString = """

SELECT distinct count(?name) ?sss where{
{SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q190379 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
} group by ?name ?sss}
UNION{
SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q190379  .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
}group by ?name ?sss}
UNION
{SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q214642 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
} group by ?name ?sss}
UNION{
SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q214642 .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
}group by ?name ?sss}

} group by ?sss
having(count(?name)>1)
order by ?sss

"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2'), ('sss', '1978')]
[('callret-0', '4'), ('sss', '1987')]
[('callret-0', '3'), ('sss', '1992')]
[('callret-0', '2'), ('sss', '2001')]
[('callret-0', '2'), ('sss', '2002')]
[('callret-0', '2'), ('sss', '2005')]
[('callret-0', '2'), ('sss', '2010')]


7

In [9]:
# Doesn't work with year(date)
queryString = """


SELECT distinct ?name ?sss where{

{SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q214642 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
} group by ?name ?sss}
UNION{
SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q214642 .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
}group by ?name ?sss}

} order by ?sss



"""

print("Results")
run_query(queryString)

Results
[('name', 'The New York Trilogy'), ('sss', '1987')]
[('name', 'In the Country of Last Things'), ('sss', '1987')]
[('name', 'The Music of Chance'), ('sss', '1990')]
[('name', 'Leviathan'), ('sss', '1992')]
[('name', 'Mr. Vertigo'), ('sss', '1994')]
[('name', 'Timbuktu'), ('sss', '1999')]
[('name', 'The Book of Illusions'), ('sss', '2002')]
[('name', 'Oracle Night'), ('sss', '2003')]
[('name', 'The Brooklyn Follies'), ('sss', '2005')]
[('name', 'Travels in the Scriptorium'), ('sss', '2006')]
[('name', 'Invisible'), ('sss', '2009')]
[('name', 'Sunset Park'), ('sss', '2010')]


12

In [10]:
queryString = """
SELECT distinct ?name ?sss where{
{SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    wd:Q190379 wdt:P800 ?obj .
    ?obj wdt:P31 ?book .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
} group by ?name ?sss}
UNION{
SELECT distinct ?name ?sss (group_concat(?genre;separator=", ") as ?genres)
where{
    ?obj wdt:P31 ?book .
    ?obj ?x wd:Q190379  .
    ?obj wdt:P136 ?genre1 .
    ?obj wdt:P577 ?pubdate .
    ?obj wdt:P1476 ?name .
    ?genre1 <http://schema.org/name> ?genre .
    filter(?book=wd:Q571 ||?book=wd:Q47461344)
    bind (substr(str(?pubdate),1,4) as ?sss)
}group by ?name ?sss}

} order by ?sss
"""

print("Results")
run_query(queryString)

Results
[('name', 'The Cement Garden'), ('sss', '1978')]
[('name', 'The Comfort of Strangers'), ('sss', '1981')]
[('name', 'The Child in Time'), ('sss', '1987')]
[('name', 'Black Dogs'), ('sss', '1992')]
[('name', 'Enduring Love'), ('sss', '1997')]
[('name', 'Amsterdam'), ('sss', '1998')]
[('name', 'Atonement'), ('sss', '2001')]
[('name', 'Saturday'), ('sss', '2005')]
[('name', 'On Chesil Beach'), ('sss', '2007')]
[('name', 'Solar'), ('sss', '2010')]
[('name', 'Sweet Tooth'), ('sss', '2012')]


11

The longest periods without publishing a book for the two authors are respectively between 1981 and 1987 for Ian McEwan and between 1994 and 1999 for Paul Auster

***

# Task 3

*Did the authors produced, acted or directed a film? If so, did they write the screenplay?*

In [28]:
# Paul Auster notable work (same for Ian McEwan)
queryString = """
SELECT ?name ?name2
where{
    wd:Q214642 wdt:P800 ?p .
    ?p wdt:P31 ?n .
    ?p <http://schema.org/name> ?name .
    ?n <http://schema.org/name> ?name2 .
    
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'The New York Trilogy'), ('name2', 'book series')]
[('name', 'The New York Trilogy'), ('name2', 'written work')]
[('name', 'The Book of Illusions'), ('name2', 'written work')]
[('name', 'Leviathan'), ('name2', 'written work')]
[('name', 'Man in the Dark'), ('name2', 'written work')]
[('name', 'The Book of Illusions'), ('name2', 'book')]


6

In [54]:
# They wrote only books (notable) aparently but let's find if the book are related to films. Dead end
queryString = """
SELECT distinct ?name 
where{
      wd:Q190379  wdt:P800 ?p .
    ?p ?x ?y .
    ?x <http://schema.org/name> ?name .
      filter(!regex(?name, "ID")) 
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'publisher')]
[('name', 'genre')]
[('name', 'title')]
[('name', 'follows')]
[('name', 'award received')]
[('name', 'image')]
[('name', 'different from')]
[('name', 'OCLC control number')]
[('name', 'instance of')]
[('name', 'language of work or name')]
[('name', 'country of origin')]
[('name', 'derivative work')]
[('name', 'author')]
[('name', 'publication date')]
[('name', 'has edition or translation')]
[('name', 'main subject')]
[('name', 'ISBN-13')]
[('name', 'ISBN-10')]


18

In [56]:
# Maybe they did films that are not related to their books. Going back to the authors properties there is "occupation" and "field of work" that can help
queryString = """
SELECT distinct ?name 
where{
      wd:Q190379 wdt:P101 ?p .
      ?p <http://schema.org/name> ?name .
} 
"""
# I checked the 2 properties for the other author and apparently it has only one of the 2 (so "occupation" is what i need)
print("Results")
run_query(queryString)

Results
Empty


0

In [58]:
# Check for Ian McEwan

queryString = """
SELECT distinct ?name 
where{
      wd:Q190379 wdt:P106 ?p .
      ?p <http://schema.org/name> ?name .
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'playwright')]
[('name', 'screenwriter')]
[('name', 'film producer')]
[('name', 'writer')]
[('name', 'author')]
[('name', 'novelist')]


6

In [59]:
# Check for Paul Auster
queryString = """
SELECT distinct ?name 
where{
      wd:Q214642 wdt:P106 ?p .
      ?p <http://schema.org/name> ?name .
} 
"""
# so they both produced and screenwritten a film (I have to check if they did both things in the same film)
print("Results")
run_query(queryString)

Results
[('name', 'essayist')]
[('name', 'linguist')]
[('name', 'university teacher')]
[('name', 'autobiographer')]
[('name', 'film director')]
[('name', 'screenwriter')]
[('name', 'translator')]
[('name', 'writer')]
[('name', 'author')]
[('name', 'poet')]
[('name', 'novelist')]


11

In [2]:
# I have to find a way to get the node "film". Writtenwork probably is the instance of a more general "work of art" 
queryString = """
SELECT distinct ?name  ?p
where{
      wd:Q47461344 ?x ?p .
      ?p <http://schema.org/name> ?name .
} 
"""
print("Results")
run_query(queryString)

Results
[('name', 'VIAF ID'), ('p', 'http://www.wikidata.org/entity/P214')]
[('name', 'language of work or name'), ('p', 'http://www.wikidata.org/entity/P407')]
[('name', 'creative work'), ('p', 'http://www.wikidata.org/entity/Q17537576')]
[('name', 'literary work'), ('p', 'http://www.wikidata.org/entity/Q7725634')]
[('name', 'version, edition, or translation'), ('p', 'http://www.wikidata.org/entity/Q3331189')]
[('name', 'book'), ('p', 'http://www.wikidata.org/entity/Q571')]
[('name', 'writing system'), ('p', 'http://www.wikidata.org/entity/Q8192')]
[('name', 'genre'), ('p', 'http://www.wikidata.org/entity/P136')]
[('name', 'Category:Written works'), ('p', 'http://www.wikidata.org/entity/Q10013893')]
[('name', 'volume'), ('p', 'http://www.wikidata.org/entity/Q105763458')]


10

In [5]:
# Creative work is probably what i need. Check for something about films and movies
queryString = """
SELECT distinct ?name  ?p
where{
    ?p ?x wd:Q17537576.
    ?p <http://schema.org/name> ?name .
    filter(regex(lcase(str(?name)),"film") || regex(lcase(str(?name)),"movie"))
} 

"""
print("Results")
run_query(queryString)

Results
[('name', 'Abraham Lincoln (film)'), ('p', 'http://www.wikidata.org/entity/Q42184819')]
[('name', 'History: The Home Movie'), ('p', 'http://www.wikidata.org/entity/Q42191130')]
[('name', 'Film'), ('p', 'http://www.wikidata.org/entity/Q42189922')]
[('name', 'Film School'), ('p', 'http://www.wikidata.org/entity/Q106995115')]
[('name', 'Painting, Photography, Film'), ('p', 'http://www.wikidata.org/entity/Q42192605')]


5

In [7]:
# properties that enter film
queryString = """
SELECT distinct ?name ?x
where{
    ?p ?x wd:Q42189922 .
    ?x <http://schema.org/name> ?name .
    
} 

"""
print("Results")
run_query(queryString)

Results
Empty


0

In [8]:
# Something is wrong, properties that exits film
queryString = """
SELECT distinct ?name ?x
where{
    wd:Q42189922 ?x ?p.
    ?x <http://schema.org/name> ?name .
} 

"""
print("Results")
run_query(queryString)

Results
[('name', 'author name string'), ('x', 'http://www.wikidata.org/prop/direct/P2093')]
[('name', 'Encyclopædia Britannica Online ID'), ('x', 'http://www.wikidata.org/prop/direct/P1417')]
[('name', 'instance of'), ('x', 'http://www.wikidata.org/prop/direct/P31')]


3

Seeing so few properties and also an ID and an author, I'm pretty sure it isn't the general node "film" but something else

In [9]:
# Definitely not what i meant
queryString = """
SELECT distinct ?x
where{
    wd:Q42189922 wdt:P2093 ?x.
} 

"""
print("Results")
run_query(queryString)

Results
[('x', 'Beckett')]


1

In [13]:
# Let's go back to creative work
queryString = """
SELECT count(distinct ?name) as ?tot  
where{
    ?p ?x wd:Q17537576.
    ?x <http://schema.org/name> ?name .
} 
"""
print("Results")
run_query(queryString)

Results
[('tot', '23')]


1

In [15]:
# Let's go back to creative work. Check the properties
queryString = """
SELECT distinct ?name ?x
where{
    ?p ?x wd:Q17537576.
    ?x <http://schema.org/name> ?name .
} 
"""
print("Results")
run_query(queryString)

Results
[('name', 'field of work'), ('x', 'http://www.wikidata.org/prop/direct/P101')]
[('name', 'product or material produced'), ('x', 'http://www.wikidata.org/prop/direct/P1056')]
[('name', 'occupation'), ('x', 'http://www.wikidata.org/prop/direct/P106')]
[('name', 'facet of'), ('x', 'http://www.wikidata.org/prop/direct/P1269')]
[('name', 'has effect'), ('x', 'http://www.wikidata.org/prop/direct/P1542')]
[('name', 'different from'), ('x', 'http://www.wikidata.org/prop/direct/P1889')]
[('name', 'interested in'), ('x', 'http://www.wikidata.org/prop/direct/P2650')]
[('name', 'subclass of'), ('x', 'http://www.wikidata.org/prop/direct/P279')]
[('name', "category's main topic"), ('x', 'http://www.wikidata.org/prop/direct/P301')]
[('name', 'instance of'), ('x', 'http://www.wikidata.org/prop/direct/P31')]
[('name', 'is a list of'), ('x', 'http://www.wikidata.org/prop/direct/P360')]
[('name', 'part of'), ('x', 'http://www.wikidata.org/prop/direct/P361')]
[('name', 'category contains'), ('x', 

23

In [16]:
# Check "instance of" and "subclass". Too many results
queryString = """
SELECT count(distinct ?name) 
where{
    ?p wdt:P31|wdt:P279 wd:Q17537576.
    ?p <http://schema.org/name> ?name .
} 
"""
print("Results")
run_query(queryString)

Results
[('callret-0', '8907')]


1

At this point I changed approach and tried to see the properties entering Ian McEwan (I know from before that "cast member" goes from the film to the person for example)

In [4]:
queryString = """
SELECT distinct ?name ?p
where{
    ?x ?p wd:Q190379 .
    ?p <http://schema.org/name> ?name .
} 
"""
print("Results")
run_query(queryString)

Results
[('name', 'winner'), ('p', 'http://www.wikidata.org/prop/direct/P1346')]
[('name', 'cast member'), ('p', 'http://www.wikidata.org/prop/direct/P161')]
[('name', 'producer'), ('p', 'http://www.wikidata.org/prop/direct/P162')]
[('name', 'spouse'), ('p', 'http://www.wikidata.org/prop/direct/P26')]
[('name', "category's main topic"), ('p', 'http://www.wikidata.org/prop/direct/P301')]
[('name', 'author'), ('p', 'http://www.wikidata.org/prop/direct/P50')]
[('name', 'screenwriter'), ('p', 'http://www.wikidata.org/prop/direct/P58')]
[('name', 'category combines topics'), ('p', 'http://www.wikidata.org/prop/direct/P971')]
[('name', 'after a work by'), ('p', 'http://www.wikidata.org/prop/direct/P1877')]
[('name', 'Wikidata property example'), ('p', 'http://www.wikidata.org/prop/direct/P1855')]


10

In [5]:
# In which film Ian Mcewan worked
queryString = """
SELECT distinct ?name 
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q190379 .
    ?x <http://schema.org/name> ?name .
} 
"""
print("Results")
run_query(queryString)

Results
[('name', 'The Good Son')]
[('name', 'Enduring Love')]
[('name', "The Ploughman's Lunch")]
[('name', 'The Comfort of Strangers')]
[('name', 'On Chesil Beach')]
[('name', 'Atonement')]
[('name', 'The Cement Garden')]
[('name', 'The Unbelievers')]
[('name', 'The Innocent')]
[('name', 'The Children Act')]


10

In [6]:
queryString = """
SELECT distinct ?name 
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q214642 .
    ?x <http://schema.org/name> ?name .
} 
"""
print("Results")
run_query(queryString)

Results
[('name', 'Blue in the Face')]
[('name', 'Act of God')]
[('name', 'The Inner Life of Martin Frost')]
[('name', 'The Music of Chance')]
[('name', 'Smoke')]
[('name', 'Lulu on the Bridge')]
[('name', 'The Center of the World')]


7

In [12]:
# ANSWER to 3
queryString = """
select ?name where{
{SELECT distinct ?name 
where{
    ?x wdt:P161|wdt:P162 wd:Q190379 .
    ?x wdt:P58 wd:Q190379 .
    ?x <http://schema.org/name> ?name .
} }
UNION{
SELECT distinct ?name 
where{
    ?x wdt:P161|wdt:P162 wd:Q214642 .
    ?x wdt:P58 wd:Q214642 .
    ?x <http://schema.org/name> ?name .
} }}
"""
print("Results")
run_query(queryString)

Results
[('name', 'Atonement')]
[('name', 'Act of God')]
[('name', 'The Inner Life of Martin Frost')]
[('name', 'The Music of Chance')]


4

***

# Task 4 

*How many films were derived from the books of these two authors?*

In [18]:
# Properties of the films
queryString = """
select distinct ?name ?y where{
{SELECT distinct ?name ?y
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q214642 .
    ?x ?y ?z .
    ?y <http://schema.org/name> ?name .
    filter(!regex(?name, "ID"))
} }
UNION{
SELECT distinct ?name ?y
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q190379 .
    ?x ?y ?z .
    ?y <http://schema.org/name> ?name .
    filter(!regex(?name, "ID"))
} }}
"""
print("Results")
run_query(queryString)

Results
[('name', 'film editor'), ('y', 'http://www.wikidata.org/prop/direct/P1040')]
[('name', 'genre'), ('y', 'http://www.wikidata.org/prop/direct/P136')]
[('name', 'title'), ('y', 'http://www.wikidata.org/prop/direct/P1476')]
[('name', 'cast member'), ('y', 'http://www.wikidata.org/prop/direct/P161')]
[('name', 'producer'), ('y', 'http://www.wikidata.org/prop/direct/P162')]
[('name', 'award received'), ('y', 'http://www.wikidata.org/prop/direct/P166')]
[('name', 'creator'), ('y', 'http://www.wikidata.org/prop/direct/P170')]
[('name', 'different from'), ('y', 'http://www.wikidata.org/prop/direct/P1889')]
[('name', 'collection'), ('y', 'http://www.wikidata.org/prop/direct/P195')]
[('name', 'FSK film rating'), ('y', 'http://www.wikidata.org/prop/direct/P1981')]
[('name', 'duration'), ('y', 'http://www.wikidata.org/prop/direct/P2047')]
[('name', 'aspect ratio'), ('y', 'http://www.wikidata.org/prop/direct/P2061')]
[('name', 'NMHH film rating'), ('y', 'http://www.wikidata.org/prop/direct/

55

In [19]:
# Most interesting properties are "based on", "main subject" and "after a work by". Check if they are really useful part 1
queryString = """
select distinct ?name ?z where{
{SELECT distinct ?name ?z
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q214642 .
    ?x wdt:P1877 ?z .
    ?z <http://schema.org/name> ?name .
    filter(!regex(?name, "ID"))
} }
UNION{
SELECT distinct ?name ?z
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q190379 .
    ?x wdt:P1877 ?z .
    ?z <http://schema.org/name> ?name .
    filter(!regex(?name, "ID"))
} }}
"""
print("Results")
run_query(queryString)

Results
[('name', 'Ian McEwan'), ('z', 'http://www.wikidata.org/entity/Q190379')]


1

In [20]:
# Most interesting properties are "based on", "main subject" and "after a work by". Check if they are really useful part 2
queryString = """
select distinct ?name ?z where{
{SELECT distinct ?name ?z
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q214642 .
    ?x wdt:P921 ?z .
    ?z <http://schema.org/name> ?name .
    filter(!regex(?name, "ID"))
} }
UNION{
SELECT distinct ?name ?z
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q190379 .
    ?x wdt:P921 ?z .
    ?z <http://schema.org/name> ?name .
    filter(!regex(?name, "ID"))
} }}
"""

print("Results")
run_query(queryString)

Results
[('name', 'prostitution'), ('z', 'http://www.wikidata.org/entity/Q36633')]
[('name', 'incest'), ('z', 'http://www.wikidata.org/entity/Q127683')]
[('name', 'World War II'), ('z', 'http://www.wikidata.org/entity/Q362')]
[('name', 'Dunkirk evacuation'), ('z', 'http://www.wikidata.org/entity/Q911972')]


4

A peculiar result, but not what I need

In [22]:
# This should be the answer to 4 but maybe they didn't work on the film based on their book so let's check that
queryString = """
select ?paulcount ?iancount where{
{SELECT count(distinct ?name) as ?paulcount
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q214642 .
    ?x wdt:P144 ?z .
    ?z <http://schema.org/name> ?name .
} }
{
SELECT count(distinct ?name) as ?iancount
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q190379 .
    ?x wdt:P144 ?z .
    ?z <http://schema.org/name> ?name .
} }}
"""

print("Results")
run_query(queryString)

Results
[('paulcount', '0'), ('iancount', '4')]


1

In [25]:
# First get the film node
queryString = """
select distinct ?name ?p  
where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q190379 .
    ?x wdt:P31 ?p .
    ?p <http://schema.org/name> ?name .
}
"""

print("Results")
run_query(queryString)

Results
[('name', 'film'), ('p', 'http://www.wikidata.org/entity/Q11424')]


1

In [27]:
# Check that the "based on" property points to the right thing

queryString = """
select ?name1 ?name2 where{
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q190379 .
    ?x wdt:P144 ?z .
    wd:Q190379 ?w ?z .
    ?w <http://schema.org/name> ?name1 .
    ?z <http://schema.org/name> ?name2 .

}
"""

print("Results")
run_query(queryString)

Results
[('name1', 'notable work'), ('name2', 'Atonement')]
[('name1', 'notable work'), ('name2', 'Atonement')]
[('name1', 'notable work'), ('name2', 'The Cement Garden')]


3

In [37]:
# I have also to specify that I'm talking about films
queryString = """
select ?paulcount ?iancount where{
{SELECT count(distinct ?name) as ?paulcount
where{
    ?x wdt:P31 wd:Q11424 . 
    ?x wdt:P144 ?z .
    wd:Q214642 wdt:P800 ?z.
    ?x <http://schema.org/name> ?name .
} }
{
SELECT count(distinct ?name) as ?iancount
where{
    ?x wdt:P31 wd:Q11424 . 
    ?x wdt:P144 ?z .
    wd:Q190379 wdt:P800 ?z .
    ?x <http://schema.org/name> ?name .
} }}
"""

print("Results")
run_query(queryString)

Results
[('paulcount', '0'), ('iancount', '2')]


1

In [45]:
# Film on which they worked in general
queryString = """
select ?paulcount ?iancount where{
{SELECT count(distinct ?name) as ?paulcount
where{
    ?x wdt:P31 wd:Q11424 .
    
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q214642 .

    ?x <http://schema.org/name> ?name .
} }
{
SELECT count(distinct ?name) as ?iancount
where{
    ?x wdt:P31 wd:Q11424 .
    
    ?x wdt:P161|wdt:P162|wdt:P58 wd:Q190379 .
    
    ?x <http://schema.org/name> ?name .
} }}
"""

print("Results")
run_query(queryString)

Results
[('paulcount', '7'), ('iancount', '10')]


1

***

# Task 5

*Which author won more literature-related awards? Have they ever being nominated for a Nobel award*


In [48]:
# Award received
queryString = """
SELECT ?award ?name
WHERE {
   wd:Q190379 wdt:P166 ?award .
   ?award <http://schema.org/name> ?name .
}
"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q56494487'), ('name', 'honorary doctor of the University of Sussex')]
[('award', 'http://www.wikidata.org/entity/Q5153503'), ('name', 'Common Wealth Award of Distinguished Service')]
[('award', 'http://www.wikidata.org/entity/Q12201477'), ('name', 'Commander of the Order of the British Empire')]
[('award', 'http://www.wikidata.org/entity/Q277787'), ('name', 'Jerusalem Prize')]
[('award', 'http://www.wikidata.org/entity/Q39477935'), ('name', 'Fellow of the Royal Society of Literature')]
[('award', 'http://www.wikidata.org/entity/Q52382875'), ('name', 'AAAS Fellow')]
[('award', 'http://www.wikidata.org/entity/Q15271633'), ('name', 'Fellow of the Royal Society of Arts')]
[('award', 'http://www.wikidata.org/entity/Q160082'), ('name', 'Booker Prize')]
[('award', 'http://www.wikidata.org/entity/Q392586'), ('name', 'James Tait Black Memorial Prize')]
[('award', 'http://www.wikidata.org/entity/Q5709276'), ('name', 'Helmerich Award')]
[('award'

15

In [51]:
# Award properties
queryString = """
SELECT distinct ?prop ?name
WHERE {
   wd:Q190379 wdt:P166 ?award .
   ?award ?prop ?y .
   ?prop <http://schema.org/name> ?name .
   filter(!regex(?name,"ID"))
}
"""

print("Results")
run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P3729'), ('name', 'next lower rank')]
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P2121'), ('name', 'prize money')]
[('prop', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('prop', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/d

28

In [55]:
# Checked some properties, what i need is as always instance of
queryString = """
SELECT distinct ?y ?name
WHERE {
   wd:Q190379 wdt:P166 ?award .
   ?award wdt:P31 ?y .
   ?y <http://schema.org/name> ?name .
   filter(!regex(?name,"ID"))
}
"""

print("Results")
run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q618779'), ('name', 'award')]
[('y', 'http://www.wikidata.org/entity/Q378427'), ('name', 'literary award')]
[('y', 'http://www.wikidata.org/entity/Q131647'), ('name', 'medal')]
[('y', 'http://www.wikidata.org/entity/Q60754876'), ('name', 'grade of an order')]
[('y', 'http://www.wikidata.org/entity/Q20006438'), ('name', 'fellowship')]


5

In [57]:
# ANSWER 5 part 1
queryString = """
select ?paulcount ?iancount where{
{SELECT count(distinct ?name) as ?paulcount
where{
     wd:Q214642 wdt:P166 ?award .
     ?award wdt:P31 wd:Q378427 .
     ?award <http://schema.org/name> ?name .
} }
{
SELECT count(distinct ?name) as ?iancount
where{
     wd:Q190379 wdt:P166 ?award .
     ?award wdt:P31 wd:Q378427 .
     ?award <http://schema.org/name> ?name .
} }}
"""

print("Results")
run_query(queryString)

Results
[('paulcount', '2'), ('iancount', '7')]


1

In [63]:
# There is nothing about nomination for prizes so let's start from the Nobel
queryString = """
select ?p ?name where
{
 ?p wdt:P31 wd:Q378427 .
 ?p <http://schema.org/name> ?name .
 filter(regex(lcase(str(?name)),"nobel"))
 filter(regex(lcase(str(?name)),"literature"))
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q37922'), ('name', 'Nobel Prize in Literature')]


1

In [67]:
# I assume that the nominated are linked to the nobel and not the other way around
queryString = """
select distinct ?x ?name where
{
  ?y ?x wd:Q37922 .
  ?x <http://schema.org/name> ?name .
}
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('x', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('x', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('x', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('x', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('x', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('x', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('x', 'http://www.wikidata.org/prop/direct/P793'), ('name', 'significant event')]
[('x', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('x', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]


10

In [68]:
# ANSWER to 5 part 2
queryString = """
ask where
{
  ?y wdt:P1411 wd:Q37922 .
  filter(?y= wd:Q190379 || ?y= wd:Q214642 )
  
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': False}